# Построение модели для прогнозирования модуля упругости при растяжении

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [2]:
#df.index = df.index.astype('int')

In [3]:
df = pd.read_csv('data_model.csv', index_col = 0)
df.sample(5)

,T1,F1,F2,F3,F4,F5,F6,T2,T3,F7,F9,F10
662.0,3.566924,1993.795963,507.319683,139.263105,23.643004,261.894153,583.323511,74.079019,2898.127599,212.109119,6.427925,50.713052
713.0,2.154655,2015.179045,623.067451,122.394948,24.683287,271.398812,204.009880,71.533710,2751.228051,198.742865,3.268975,67.038824
683.0,1.085399,1845.739087,676.621615,95.375196,19.319844,313.184981,628.368773,78.895826,3409.525741,175.767919,7.068781,64.552829
950.0,3.353945,1964.801437,608.673260,163.985672,23.393889,276.710021,714.818313,71.237408,3321.035587,197.328325,6.270970,60.483248
494.0,1.512672,1994.132101,908.615893,53.266747,21.588737,368.772634,285.543446,71.444581,2350.243870,225.167976,6.063359,80.743005


In [4]:
df.columns

Index(['T1', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'T2', 'T3', 'F7', 'F9',
       'F10'],
      dtype='object')

1. Упражнения на всей выборке

In [7]:
target = ['T2']
features = ['T1', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'T2', 'F7', 'F9', 'F10']

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state = 5)

#res= clf._predict_proba_lr(X_test,Y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(655, 11)
(281, 11)
(655, 1)
(281, 1)


In [8]:
linear = LinearRegression()
linear.fit(X_train, y_train)

y_pred = linear.predict(X_test)
mse_linear = mean_squared_error(y_pred, y_test)

print(mse_linear)

2.1560326519919466e-28


In [32]:
print(linear.coef_)

[[ 4.17114985e-02 -3.01502340e-03  5.51650651e-04 -8.93464635e-03
   5.30423197e-02 -9.08842063e-05  1.29980323e-04  3.67129137e-05
   2.72638750e-03 -1.69385681e-02  1.32012220e-03]]


In [33]:
lambda_values = [0.000001, 0.0001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5]

for lambda_val in lambda_values:
    lasso = Lasso(lambda_val)
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    mse_lasso = mean_squared_error(y_pred, y_test)
    print(('Lasso MSE with Lambda = {} is {}').format(lambda_val, mse_lasso))

Lasso MSE with Lambda = 1e-06 is 10.449171673543068
Lasso MSE with Lambda = 0.0001 is 10.449087734627977
Lasso MSE with Lambda = 0.005 is 10.444985812292922
Lasso MSE with Lambda = 0.01 is 10.440905179476687
Lasso MSE with Lambda = 0.05 is 10.42757905728457
Lasso MSE with Lambda = 0.1 is 10.4352783741403
Lasso MSE with Lambda = 0.2 is 10.451095605153833
Lasso MSE with Lambda = 0.3 is 10.466830191673312
Lasso MSE with Lambda = 0.5 is 10.459502390120019


In [34]:
print(lasso.coef_)

[ 0.00000000e+00 -2.81172868e-03  5.35995064e-04 -8.15733121e-03
  0.00000000e+00 -0.00000000e+00  1.10376984e-04  4.23908885e-05
  2.60378361e-03 -0.00000000e+00  0.00000000e+00]


In [35]:
lambda_values = [0.000001, 0.0001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.3, 0.5]

for lambda_val in lambda_values:
    ridge = Ridge(lambda_val)
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    mse_ridge = mean_squared_error(y_pred, y_test)
    print(('Ridge MSE with Lambda = {} is {}').format(lambda_val, mse_ridge))

Ridge MSE with Lambda = 1e-06 is 10.449172521547883
Ridge MSE with Lambda = 0.0001 is 10.449172516583392
Ridge MSE with Lambda = 0.005 is 10.449172270869289
Ridge MSE with Lambda = 0.01 is 10.449172020146989
Ridge MSE with Lambda = 0.05 is 10.449170014600522
Ridge MSE with Lambda = 0.1 is 10.44916750824709
Ridge MSE with Lambda = 0.2 is 10.449162497471619
Ridge MSE with Lambda = 0.3 is 10.449157489269924
Ridge MSE with Lambda = 0.5 is 10.449147480581164


In [36]:
print(ridge.coef_)

[[ 4.16684478e-02 -3.01500954e-03  5.51651878e-04 -8.93460269e-03
   5.30346766e-02 -9.09378884e-05  1.29981318e-04  3.67135199e-05
   2.72642678e-03 -1.69362159e-02  1.32013152e-03]]


In [55]:
df_1 = df.drop(['T2'], axis = 1)
X, y = df_1.values, df['T2'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state = 42)

#res= clf._predict_proba_lr(X_test,Y_test)

X_train.shape

(655, 11)

In [56]:
scaler = StandardScaler()

In [57]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [58]:
X_train_scaled.shape

(655, 11)

In [59]:
X_test_scaled.shape

(281, 11)

In [60]:
def train_validate_report(model, X_train_scaled, y_train, X_test_scaled, y_test, feature_names):
    
    model.fit(X_train_scaled, y_train)
    
    print('MAE:', mean_absolute_error(y_test, model.predict(X_test_scaled)))
    print('MSE:', mean_squared_error(y_test, model.predict(X_test_scaled)))
    print('RMSE:', np.sqrt(mean_squared_error(y_test, model.predict(X_test_scaled))))
    
    print(pd.DataFrame(model.coef_, feature_names, columns = ['coef']).sort_values(
        by = 'coef', ascending = False))

In [61]:
lasso = Lasso(random_state = 17)
ridge = Ridge(random_state = 17)
lasso_cv = LassoCV(random_state = 17)
ridge_cv = RidgeCV()

In [62]:
train_validate_report(lasso, X_train_scaled, y_train, X_test_scaled, y_test, 
                      feature_names = df.columns[:-1])

MAE: 2.3986344930417918
MSE: 8.765901841522709
RMSE: 2.960726573245613
    coef
T1  -0.0
F1  -0.0
F2   0.0
F3  -0.0
F4   0.0
F5   0.0
F6   0.0
T2  -0.0
T3   0.0
F7  -0.0
F9   0.0


In [63]:
train_validate_report(ridge, X_train_scaled, y_train, X_test_scaled, y_test, feature_names = df.columns[:-1])

MAE: 2.4182609859719615
MSE: 8.776976606130868
RMSE: 2.9625962610742063
        coef
F2  0.174437
F4  0.169266
F6  0.135203
T3  0.127073
F9  0.096131
F5  0.014689
F1 -0.061166
T2 -0.064422
F7 -0.072487
T1 -0.111564
F3 -0.168502


In [64]:
train_validate_report(lasso_cv, X_train_scaled, y_train, X_test_scaled, y_test, feature_names = df.columns[:-1])

MAE: 2.3986344930417918
MSE: 8.765901841522709
RMSE: 2.960726573245613
            coef
F2  6.508788e-17
T1 -0.000000e+00
F1 -0.000000e+00
F3 -0.000000e+00
F4  0.000000e+00
F5  0.000000e+00
F6  0.000000e+00
T2 -0.000000e+00
T3  0.000000e+00
F7 -0.000000e+00
F9  0.000000e+00


In [65]:
train_validate_report(ridge_cv, X_train_scaled, y_train, X_test_scaled, y_test, feature_names = df.columns[:-1])

MAE: 2.417859597558593
MSE: 8.774790315131943
RMSE: 2.9622272558215283
        coef
F2  0.171922
F4  0.166719
F6  0.133023
T3  0.125163
F9  0.094671
F5  0.014611
F1 -0.059813
T2 -0.063016
F7 -0.071267
T1 -0.109882
F3 -0.165828


2. На важных признаках

In [47]:
df_1 = df.drop(['T2', 'F1', 'F2', 'T1', 'F5', 'T3', 'F10'], axis = 1)
X, y = df_1.values, df['T2'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state = 42)

#res= clf._predict_proba_lr(X_test,Y_test)

X_train.shape

(655, 5)

In [48]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [49]:
def train_validate_report(model, X_train_scaled, y_train, X_test_scaled, y_test, feature_names):
    
    model.fit(X_train_scaled, y_train)
    
    print('MAE:', mean_absolute_error(y_test, model.predict(X_test_scaled)))
    print('MSE:', mean_squared_error(y_test, model.predict(X_test_scaled)))
    print('RMSE:', np.sqrt(mean_squared_error(y_test, model.predict(X_test_scaled))))
    
    print(pd.DataFrame(model.coef_, feature_names, columns = ['coef']).sort_values(by = 'coef', ascending = False))

In [50]:
train_validate_report(ridge_cv, X_train_scaled, y_train, X_test_scaled, y_test, feature_names = df_1.columns[:])

MAE: 2.3920933436025504
MSE: 8.666974694942374
RMSE: 2.9439726043124743
        coef
F4  0.153404
F6  0.127902
F7  0.120786
F9 -0.065374
F3 -0.153397


Второй вариант

In [51]:
df_2 = df.drop(['T2', 'F1', 'T1', 'F5', 'F10'], axis = 1)
X, y = df_2.values, df['T2'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,random_state = 42)

#res= clf._predict_proba_lr(X_test,Y_test)

X_train.shape

(655, 7)

In [52]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [53]:
def train_validate_report(model, X_train_scaled, y_train, X_test_scaled, y_test, feature_names):
    
    model.fit(X_train_scaled, y_train)
    
    print('MAE:', mean_absolute_error(y_test, model.predict(X_test_scaled)))
    print('MSE:', mean_squared_error(y_test, model.predict(X_test_scaled)))
    print('RMSE:', np.sqrt(mean_squared_error(y_test, model.predict(X_test_scaled))))
    
    print(pd.DataFrame(model.coef_, feature_names, columns = ['coef']).sort_values(by = 'coef', ascending = False))

In [54]:
train_validate_report(ridge_cv, X_train_scaled, y_train, X_test_scaled, y_test, feature_names = df_2.columns[:])

MAE: 2.4091779168014202
MSE: 8.765646670396103
RMSE: 2.9606834802788535
        coef
F2  0.173727
F4  0.159520
F6  0.125949
F7  0.118288
T3 -0.058801
F9 -0.075542
F3 -0.161315
